Name: Sathish Kumar Rajendiran
Task: Week7: 7.4 MongoDB
Date: 8/12/2020

In [1]:
#import libraries
from urllib import request
import json
import pymongo
from pymongo import MongoClient
from datetime import datetime


In [2]:
# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

In [3]:
try:
    # reponse 
    response = request.urlopen(earthquake_url).read().decode('utf8')
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
# the url request was successful - convert the response to a string
    eq_json = json.loads(response)


In [4]:
# from the json dictionary we get the title to print
title = eq_json['metadata']['title']
print('Collected data from', title,'\n')
#  and we get the list of earthquakes
quakelist = eq_json['features']
print('quake list', len(quakelist))

Collected data from USGS Significant Earthquakes, Past Month 

quake list 11


In [11]:
# Connection to Mongo DB
try:
    client = MongoClient('localhost', 27017)
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e )

else:
    
    # use database named usgs or create it if not there already
    eqdb = client.usgs
    # create collection named earthquakes or create it if not there already
    quakecoll = eqdb.earthquakes
    # add all the earthquakes to the list
    quakecoll.insert_many(quakelist)
    print("Added", len(quakelist), "to earthquakes collection in usgs database")
    # close the database connection
    client.close()

Connected successfully!!!
Added 11 to earthquakes collection in usgs database


In [6]:
#search the first item from the collection
quakecoll.find_one()

{'_id': ObjectId('5f35d17d260d01ce7702bc63'),
 'type': 'Feature',
 'properties': {'mag': 6,
  'place': '66 km ESE of Vikindu, Tanzania',
  'time': 1597252395896,
  'updated': 1597353013040,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000bdg8',
  'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us6000bdg8.geojson',
  'felt': 211,
  'cdi': 4.3,
  'mmi': 5.361,
  'alert': 'green',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 645,
  'net': 'us',
  'code': '6000bdg8',
  'ids': ',us6000bdg8,',
  'sources': ',us,',
  'types': ',dyfi,ground-failure,losspager,moment-tensor,origin,phase-data,shakemap,',
  'nst': None,
  'dmin': 5.239,
  'rms': 0.98,
  'gap': 26,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 6.0 - 66 km ESE of Vikindu, Tanzania'},
 'geometry': {'type': 'Point', 'coordinates': [39.8073, -7.3283, 15.49]},
 'id': 'us6000bdg8'}

In [ ]:
results = quakecoll.find()
for result in results:
    place = result['properties']['place']
    unix_time_mil = result['properties']['time'] # select unix timestamp in milliseconds
    unix_time = unix_time_mil / 1000     # convert to unix in seconds
    date = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
    time = datetime.fromtimestamp(unix_time).strftime('%H:%M:%S')
    # print the place ,date and time, with a label to show the time is in UTC
    print('Place: ',place,' Date: ',date,'  Time: ',time )
   

In [10]:
#list db names
eqdb.list_collection_names()

[]

In [9]:
#drop db
eqdb.earthquakes.drop()

In [8]:
#empty database
eqdb.list_collection_names()

['earthquakes']